In [1]:
from sympy import sin, cos, pi, symbols, Eq
from sympy import solve, Matrix, sqrt, diff

## (a) Parametrització 

In [2]:
x, y, z, t = symbols('x y z t', real=True)

In [26]:
eqs = [Eq(x**2 + y**2, 1), Eq(y**2 + z**2, 1)]; eqs

[x**2 + y**2 == 1, y**2 + z**2 == 1]

De la primera equació es veu que podem usar $x = \cos t$ i $y = \sin t$ per a $t\in[0,2\pi]$ com de costum.
Restant les 2 equacions obtenim que $z=\pm x$.
Alternativement es pot substituir $y$ a la segona i deduiriem també que $z = \pm\cos t$

In [4]:
r1 = {x:cos(t), y:sin(t),z: -cos(t)}
r2 = {x:cos(t), y:sin(t),z: cos(t)}

Les dues corbes parametritzades compleixen les equacions:

In [5]:
for r in (r1,r2):
    assert all(e.subs(r).simplify() for e in eqs)

## (b) Punts d'intersecció

Igualem les components de $r_1(t)=r_2(s)$ i eliminem la solució trivial $s=t$.

In [25]:
s = symbols('s', real=True)
eqi = []
for c in (x,y,z):
    e = Eq(c.subs(r1), c.subs(r2).subs(t,s))
    eqi.append(e)
eqi

[cos(t) == cos(s), sin(t) == sin(s), -cos(t) == cos(s)]

Les dues primeres equacions conjuntament forcen que $s=t$, però llavors la tercera diu $-\cos t = \cos t$.
Això només passarà si $\cos t = 0$, és a dir si $t= \pi/2, 3\pi/2$.

In [8]:
ti = (pi/2, 3*pi/2)
punt = Matrix([x,y,z])
p1 = punt.subs(r1)
p2 = punt.subs(r2)

for u in ti: 
    assert p1.subs(t,u) == p2.subs(t,u)
    print tuple(p1.subs(t,u))

(0, 1, 0)
(0, -1, 0)


## (c) Extrems dels radis curvatura

Observem que les dues corbes són isomètriques (via $z\mapsto -z$), per tant només cal estudiar-ne una.

$\vec r'(t)$ i $\vec r''(t)$ són

In [9]:
rd = p2.diff(t); print rd
rdd = rd.diff(t); print rdd

Matrix([[-sin(t)], [cos(t)], [-sin(t)]])
Matrix([[-cos(t)], [-sin(t)], [-cos(t)]])


La norma del vector tangent és (_celeritat_):

In [10]:
rd.norm()

sqrt(2*sin(t)**2 + cos(t)**2)

In [11]:
_.simplify()

sqrt(sin(t)**2 + 1)

In [12]:
rd.cross(rdd)

Matrix([
[-sin(t)**2 - cos(t)**2],
[                     0],
[ sin(t)**2 + cos(t)**2]])

In [13]:
_.norm().simplify()

sqrt(2)

In [14]:
_/rd.norm()**3

sqrt(2)/(2*sin(t)**2 + cos(t)**2)**(3/2)

In [15]:
kappa = _.simplify(); kappa

sqrt(2)/(sin(t)**2 + 1)**(3/2)

el _radi de curvatura_ és $1/\kappa$:

In [16]:
1/kappa

sqrt(2)*(sin(t)**2 + 1)**(3/2)/2

D'aquí és veu que els extrems seran quan $\sin t=0$ o $\sin t =\pm1$:

In [17]:
[1/kappa.subs(t,u) for u in (0,pi/2)]

[sqrt(2)/2, 2]

* Però també es podria derivar i igualar a zero:

In [19]:
(1/kappa).diff(t).simplify()

3*sqrt(-cos(2*t) + 3)*sin(2*t)/4

... que només s'anul·la quan $t=k\pi/2$, per a $k=0,1,2,3$.

## (d) Digueu si les corbes són planes

* Una manera directa de fer-ho és observar que $r_1$ és dins del pla $x + z=0$ i que $r_2$ és dins de $x -z = 0$.

* Una altra manera (molt més llarga) és calcular-ne la _torsió_ i comprovar que doni zero.

In [20]:
rddd = rdd.diff(t); rddd

Matrix([
[ sin(t)],
[-cos(t)],
[ sin(t)]])

In [21]:
Matrix([u.transpose() for u in (rd,rdd,rdd)])

Matrix([
[-sin(t),  cos(t), -sin(t)],
[-cos(t), -sin(t), -cos(t)],
[-cos(t), -sin(t), -cos(t)]])

In [22]:
_.det()

0

Aquest determinant (que és igual a $(\vec r'\times\vec r'')\cdot \vec r'''$) és el que surt al numerador de la fórmula de $\tau$. Per tant la corba és plana.

## (e) Radi de la bola més grossa

### Per Lagrange

Es tracta de trobar quina és la distància mínima a l'origen de coordenades d'un punt de la intersecció dels dos cilindres.
Una bola centrada a l'origen amb radi més gran contindrà necessàriament aquest punt i, per tant, ja no serà vàlida.

Plantegem la _Laplaciana_ amb dues condicions:
$$\lambda \left(x^{2} + y^{2} - 1\right) + \mu \left(y^{2} + z^{2} - 1\right) + x^{2} + y^{2} + z^{2}$$

In [27]:
l,m = symbols('lambda mu')
from sympy import latex

L = x**2 + y**2 + z**2 + sum(g*(e.lhs - e.rhs) for g,e in zip((l,m), eqs)); L

lambda*(x**2 + y**2 - 1) + mu*(y**2 + z**2 - 1) + x**2 + y**2 + z**2

El sistema que s'ha de resoldre és $\nabla L = \vec0$:

In [39]:
sols = []
leqs = [Eq(L.diff(v).factor()) for v in (x,y,z,l,m)]; leqs

[2*x*(lambda + 1) == 0,
 2*y*(lambda + mu + 1) == 0,
 2*z*(mu + 1) == 0,
 x**2 + y**2 - 1 == 0,
 y**2 + z**2 - 1 == 0]

la tercera equació ens diu que $z=0$ o $\mu = -1$.

Anem cas per cas:

* Cas $z=0$:

In [40]:
sol = {z:0}
ez = [e.subs(sol) for e in leqs if e.subs(sol) != True]; ez

[2*x*(lambda + 1) == 0,
 2*y*(lambda + mu + 1) == 0,
 x**2 + y**2 - 1 == 0,
 y**2 - 1 == 0]

Resolem per a $y$ a la darrera equació:

In [41]:
ys = solve(ez[-1],y, dict=True); ys

[{y: -1}, {y: 1}]

In [42]:
for u in ys: 
    sol.update(u)
    print sol,[e.subs(sol) for e in ez if e.subs(sol) != True]

{y: -1, z: 0} [2*x*(lambda + 1) == 0, -2*lambda - 2*mu - 2 == 0, x**2 == 0]
{y: 1, z: 0} [2*x*(lambda + 1) == 0, 2*lambda + 2*mu + 2 == 0, x**2 == 0]


Les dues alternatives forcen que $x=0$ a la darrera equació:

In [43]:
sol[x] = 0
for u in ys: 
    sol.update(u)
    sols.append(dict(sol))
    print tuple(punt.subs(sol))
    print [e.subs(sol) for e in ez if e.subs(sol) != True]

(0, -1, 0)
[-2*lambda - 2*mu - 2 == 0]
(0, 1, 0)
[2*lambda + 2*mu + 2 == 0]


i les dues alternatives són possibles. Això ens dóna 2 punts.

* Cas $\mu = -1$:

In [44]:
sol = {m: -1}
em = [e.subs(sol) for e in leqs if e.subs(sol) != True]; em

[2*x*(lambda + 1) == 0,
 2*lambda*y == 0,
 x**2 + y**2 - 1 == 0,
 y**2 + z**2 - 1 == 0]

Observant les dues primeres veiem que $x=0$ o $\lambda=-1$ combinat amb $\lambda=0$ o $y=0$.
Examinant les alternatives de $\lambda$:

* Cas $\lambda=0$, $\mu = -1$:

In [45]:
sol.update({l: 0})
[e.subs(sol) for e in leqs if e.subs(sol) != True]

[2*x == 0, x**2 + y**2 - 1 == 0, y**2 + z**2 - 1 == 0]

d'aquí es deduix que $x=0$:

In [46]:
sol.update({x: 0})
set(e.subs(sol) for e in leqs if e.subs(sol) != True)

{y**2 - 1 == 0, y**2 + z**2 - 1 == 0}

Ara resulta que $y^2=1$, amb la qual cosa $z=0$:

In [47]:
sol.update({z:0})
set(e.subs(sol) for e in leqs if e.subs(sol) != True)

{y**2 - 1 == 0}

In [48]:
solve(_, dict=True)

[{y: -1}, {y: 1}]

i obtenim els punts següents (que, de fet, ja havien sortit):

In [49]:
for u in _:
    sol.update(u)
    print tuple(punt.subs(sol))
    sols.append(dict(sol))

(0, -1, 0)
(0, 1, 0)


* Cas $\lambda = \mu = -1$:

In [50]:
sol = {m: -1, l:-1}
set(e.subs(sol) for e in leqs if e.subs(sol) != True)

{-2*y == 0, y**2 + z**2 - 1 == 0, x**2 + y**2 - 1 == 0}

Ara veiem que per força $y=0$:

In [51]:
sol[y] = 0
set(e.subs(sol) for e in leqs if e.subs(sol) != True)

{x**2 - 1 == 0, z**2 - 1 == 0}

In [52]:
solve(_, dict=True)

[{x: -1, z: -1}, {x: -1, z: 1}, {x: 1, z: -1}, {x: 1, z: 1}]

Això ens porta a $x=\pm1$, $z=\pm1$ i obtenim els punts:

In [53]:
for u in _:
    sol.update(u)
    print tuple(punt.subs(sol))
    sols.append(dict(sol))

(-1, 0, -1)
(-1, 0, 1)
(1, 0, -1)
(1, 0, 1)


ja hem mirat totes les combinacions.
Ara el que queda és avaluar la distància a cada punt possible i quedar-nos amb el valor més petit:

In [54]:
f = punt.norm(); f

sqrt(x**2 + y**2 + z**2)

In [55]:
for s in sols:
    d = f.subs(s)
    print d, tuple(punt.subs(s))

1 (0, -1, 0)
1 (0, 1, 0)
1 (0, -1, 0)
1 (0, 1, 0)
sqrt(2) (-1, 0, -1)
sqrt(2) (-1, 0, 1)
sqrt(2) (1, 0, -1)
sqrt(2) (1, 0, 1)


### Directament

Aquesta alternativa no és legal ja que es demanava que és fes per _Lagrange_ però correcta:

In [57]:
d2 = punt.subs(r1).norm()**2; d2

sin(t)**2 + 2*cos(t)**2

In [58]:
d2.diff(t)

-2*sin(t)*cos(t)

In [59]:
for u in solve(_,t):
    print u, tuple(punt.subs(r1).subs(t,u)), sqrt(d2.subs(t,u))

0 (1, 0, -1) sqrt(2)
pi/2 (0, 1, 0) 1
pi (-1, 0, 1) sqrt(2)
3*pi/2 (0, -1, 0) 1
